
# mergeFlairResults.ipynb
## Marcus Viscardi,    February 27, 2022
Just a quick script to load up the flair results from my 5 total vs polyA RNA libraries I am writing about. Josh wants this file to try to feed it to flair

In [88]:
import pandas as pd
import numpy as np
pd.set_option('display.width', 400)
pd.set_option('display.max_columns', None)

from pprint import pprint

from nanoporePipelineCommon import pick_libs_return_paths_dict

print('imports done.')

imports done.


In [91]:
libraries = ['polyA', 'polyA2', 'polyA3', 'totalRNA2', 'totalRNA3']

lib_paths_dict = pick_libs_return_paths_dict(libraries, file_suffix='.tsv', file_midfix='compressedOnTranscripts_simple')
print()
pprint(lib_paths_dict)

Looking for file for totalRNA2, at /data16/marcus/working/210720_nanoporeRun_totalRNA_0639_L3_replicate/output_dir/merge_files/*compressedOnTranscripts_simple.tsv... File Found.
Looking for file for polyA, at /data16/marcus/working/210528_NanoporeRun_0639_L3s/output_dir/merge_files/*compressedOnTranscripts_simple.tsv... File Found.
Looking for file for polyA2, at /data16/marcus/working/210719_nanoporeRun_polyA_0639_L3_replicate/output_dir/merge_files/*compressedOnTranscripts_simple.tsv... File Found.
Looking for file for polyA3, at /data16/marcus/working/220131_nanoporeRun_polyA_0639_L3_third/output_dir/merge_files/*compressedOnTranscripts_simple.tsv... File Found.
Looking for file for totalRNA3, at /data16/marcus/working/220131_nanoporeRun_totalRNA_0639_L3_third/output_dir/merge_files/*compressedOnTranscripts_simple.tsv... File Found.

{'polyA': '/data16/marcus/working/210528_NanoporeRun_0639_L3s/output_dir/merge_files/220227_compressedOnTranscripts_simple.tsv',
 'polyA2': '/data16/ma

In [133]:
libs_df_dict = {lib_name: pd.read_csv(lib_path, sep="\t")[["gene_id", "transcript_id", "transcript_hits"]] for lib_name, lib_path in lib_paths_dict.items()}

for lib_name, lib_df in libs_df_dict.items():
    norm_factor = lib_df[f"transcript_hits"].sum()
    # Turn the total number of read hits into the 'million of read hits'
    rpm_norm_factor = norm_factor / 1000000
    # For each library divide gene_hits by the rpm norm factor to get rpm
    lib_df["transcript_rpm"] = lib_df[f"transcript_hits"] / rpm_norm_factor
    print(f"{lib_name}: hits sum={norm_factor}, rpm norm fact={rpm_norm_factor}")
    print(lib_df.query("transcript_id == 'W09G10.9.1'"))

totalRNA2: hits sum=549439, rpm norm fact=0.549439
             gene_id transcript_id  transcript_hits  transcript_rpm
3277  WBGene00268212    W09G10.9.1               19       34.580727
polyA: hits sum=1291774, rpm norm fact=1.291774
             gene_id transcript_id  transcript_hits  transcript_rpm
2495  WBGene00000197    W09G10.9.1               62       47.996012
polyA2: hits sum=1069960, rpm norm fact=1.06996
             gene_id transcript_id  transcript_hits  transcript_rpm
2773  WBGene00271830    W09G10.9.1               45       42.057647
polyA3: hits sum=1388781, rpm norm fact=1.388781
             gene_id transcript_id  transcript_hits  transcript_rpm
4462  WBGene00271830    W09G10.9.1               34       24.481902
totalRNA3: hits sum=957438, rpm norm fact=0.957438
     gene_id transcript_id  transcript_hits  transcript_rpm
3876     NaN    W09G10.9.1               30       31.333622


In [131]:
libs_df_dict2 = {lib_name: lib_df.set_index(["gene_id", "transcript_id"]).copy() for lib_name, lib_df in libs_df_dict.items()}
long_df2 = pd.concat(libs_df_dict2.values(), keys=libs_df_dict2.keys(), names=["lib"], axis=1)
long_df2.columns.rename("val", level=1, inplace=True)
long_df2.sort_values(["val", "lib"], axis=1, inplace=True)

long_df2.columns = ['_'.join(col).strip('_') for col in long_df2.columns.values]
long_df2.sort_values("polyA_transcript_rpm", ascending=False, inplace=True)
long_df2.query("transcript_id == 'W09G10.9.1'")

,,polyA_transcript_hits,polyA2_transcript_hits,polyA3_transcript_hits,totalRNA2_transcript_hits,totalRNA3_transcript_hits,polyA_transcript_rpm,polyA2_transcript_rpm,polyA3_transcript_rpm,totalRNA2_transcript_rpm,totalRNA3_transcript_rpm
gene_id,transcript_id,,,,,,,,,,
WBGene00000197,W09G10.9.1,62.0,NaN,NaN,NaN,NaN,47.996012,NaN,NaN,NaN,NaN
WBGene00268212,W09G10.9.1,NaN,NaN,NaN,19.0,30.0,NaN,NaN,NaN,34.580727,31.333622
WBGene00271830,W09G10.9.1,NaN,45.0,34.0,NaN,30.0,NaN,42.057647,24.481902,NaN,31.333622
NaN,W09G10.9.1,NaN,NaN,NaN,NaN,30.0,NaN,NaN,NaN,NaN,31.333622


In [130]:
long_df2.to_csv("./220227_mergingFeatureCounts.tsv", sep="\t")
pd.read_csv("./220227_mergingFeatureCounts.tsv", sep="\t")

,gene_id,transcript_id,polyA_transcript_hits,polyA2_transcript_hits,polyA3_transcript_hits,totalRNA2_transcript_hits,totalRNA3_transcript_hits,polyA_transcript_rpm,polyA2_transcript_rpm,polyA3_transcript_rpm,totalRNA2_transcript_rpm,totalRNA3_transcript_rpm
0,WBGene00010965,MTCE.31.1,29356.0,24475.0,12339.0,11379.0,10385.0,22725.337404,22874.686904,8884.770169,20710.215329,10846.655345
1,WBGene00010962,MTCE.23.1,17505.0,14612.0,8199.0,7352.0,8256.0,13551.132009,13656.585293,5903.738602,13380.921267,8623.012665
2,WBGene00010960,MTCE.12.1,16717.0,16791.0,7726.0,9302.0,9289.0,12941.118183,15693.110023,5563.152146,16929.995869,9701.933702
3,WBGene00010964,MTCE.26.1,15997.0,17803.0,7201.0,11832.0,12592.0,12383.745144,16638.939773,5185.122780,21534.692659,13151.765441
4,WBGene00021350,Y37E3.8a.1,9971.0,7296.0,12163.0,3378.0,7509.0,7718.842460,6818.946503,8758.040325,6148.089233,7842.805487
...,...,...,...,...,...,...,...,...,...,...,...,...
27379,NaN,ZK971.1.1,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,6.266724
27380,NaN,ZK973.2.2,NaN,7.0,NaN,5.0,NaN,NaN,6.542301,NaN,9.100191,NaN
27381,NaN,ZK973.3.1,NaN,NaN,NaN,14.0,NaN,NaN,NaN,NaN,25.480536,NaN
27382,NaN,ZK973.9.1,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,10.920230,NaN
